In [ ]:
# Dinc Erduran 262999

In [25]:
import requests
import os
import nltk

def remove_non_ascii(text):
    return ''.join([i if ord(i) < 128 else '_' for i in text])

In [26]:
# 1a
# if you use parse instead of query you get a nicer format returned
url = "https://en.wikipedia.org/w/api.php?action=parse&prop=links&page=web_design&format=json"
r = requests.get(url)

In [27]:
# 1b

In [ ]:
for link in r.json()["parse"]["links"]:
    if link["ns"] == 0 and "exists" in link:
        file = open(link["*"]+".txt","w") 
        tmp = requests.get("https://en.wikipedia.org/w/api.php?format=json&action=query&prop=extracts&exintro=1&explaintext=1&titles="+link["*"].replace(" ","_"))
        file.write(remove_non_ascii(list(tmp.json()["query"]["pages"].values())[0]["extract"]))
        file.close() 

In [ ]:
# 2a
nltk.download('stopwords')
stops = set(nltk.corpus.stopwords.words("english")) 

In [ ]:
for filename in os.listdir(os.getcwd()):
    if filename.endswith(".txt"):
        orig = open(filename, "r")
        nostops = open(filename[0:-4]+".nstxt", "w")
        
        filtered_words = [word for word in orig.read().lower().replace(",","").replace(".","").replace("(","").replace(")","").split(" ") if word not in stops]
        
        nostops.write(" ".join(filtered_words))
        
        orig.close()
        nostops.close()

In [ ]:
#2b
#both have trouble with singular-plural
#stemmer = nltk.stem.snowball.SnowballStemmer("english")
stemmer = nltk.stem.porter.PorterStemmer()

In [ ]:
for filename in os.listdir(os.getcwd()):
    if filename.endswith(".nstxt"):
        orig = open(filename, "r")
        stemmed = open(filename[0:-6]+".stxt", "w")
        
        words = orig.read().split(" ")
        swords = []
        for word in words:
            swords.append(stemmer.stem(word))
            
        
        stemmed.write(" ".join(swords))
        
        orig.close()
        stemmed.close()

In [9]:
#2.3
import glob
import itertools

import os
documents = []
titles = []
for filename in os.listdir(os.getcwd()):
    if filename.endswith(".txt"):
        titles.append(os.path.basename(filename))
        tmp = open(filename,"r")
        documents.append(tmp.read())
        tmp.close()
documents.insert(0, "web development design")
titles.insert(0, "query")

from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

vectorizer = CountVectorizer()
term_vectors = vectorizer.fit_transform(documents)


tdm2 = np.array(term_vectors.toarray())
# 2.3) if it has to be term as i doc as j
print(tdm2.transpose())


[[0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [10]:
from sklearn.feature_extraction.text import TfidfTransformer

transformer = TfidfTransformer(use_idf=False, smooth_idf=False)
# for 3.1a)
tf = transformer.fit_transform(term_vectors.toarray())



transformer = TfidfTransformer(use_idf=True, smooth_idf=False)
# for 3.1b)
tfidf = transformer.fit_transform(term_vectors.toarray())

from sklearn.metrics.pairwise import linear_kernel

# added the query "web development design" as the first row 
cosine_similarities_tf = linear_kernel(tf[0:1], tf).flatten()
top5_tf = cosine_similarities_tf.argsort()[:-7:-1]
print("tf cosine similarity top 5")
for i in range(0, 6):
  print (titles[top5_tf[i]])


cosine_similarities_tfidf = linear_kernel(tfidf[0:1], tfidf).flatten()
top5_tfidf = cosine_similarities_tfidf.argsort()[:-7:-1]
print("tf idf cosine similarity top 5")
for i in range(0, 6):
  print (titles[top5_tfidf[i]])


tf cosine similarity top 5
query
Adaptive web design.txt
Design education.txt
Web development.txt
Tableless web design.txt
Web developer.txt
tf idf cosine similarity top 5
query
Web development.txt
Web developer.txt
Adaptive web design.txt
Web engineering.txt
Web 2.0.txt


In [13]:
#3.2
import itertools
vocab = vectorizer.get_feature_names()
vocdict = dict(itertools.zip_longest(vocab, range(len(vocab))))

i = 0
probs = {}
for row in term_vectors.toarray():
  total = (1.0/max(1, np.sum(row)))
  p = row[vocdict['web']]*total + row[vocdict['development']]*total + row[vocdict['design']]*total
  probs[i] = p
  i += 1

import operator
sorted_probs = sorted(probs.items(), key=operator.itemgetter(1), reverse=True)

for i in range(0, 6):
  print (titles[sorted_probs[i][0]])

query
Adaptive web design.txt
Continuous design.txt
Web developer.txt
Design education.txt
Empathic design.txt
